In [5]:
!py -m pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
      --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
      --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
      --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
     - -------------------------------------- 0.0/1.5 MB 115.5 kB/s eta 0:00:13
     - -------------------------------------- 0.1/1.5 MB 172.4 kB/s eta 0:00:09
     -- ------------------------------------- 0.1/1.5 MB 228.2 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/1.5 MB 228.2 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/1.5 MB 228.2 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/1.5 MB 204.8 kB/s eta 0:00:07
     --- ------------------------------------ 0.1/1.5 MB 2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\Vaidik\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [91]:
import numpy as np
import pandas as pd
# regular expression library, useful for searching a text in a document
import re
# nltk = natural language toolkit
import nltk
from nltk.corpus import stopwords    # those words that do not have much value to our paragraph or text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # to convert text into feature vectors/numbers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [92]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vaidik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### printing the stopwords

In [93]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing

In [109]:
news_dataset = pd.read_csv('train.csv')

In [110]:
news_dataset.shape

(20800, 5)

In [111]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [112]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [113]:
# replacing the missing values with empty strings
news_dataset = news_dataset.fillna('')

In [114]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [115]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [116]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [117]:
# seperating the data and label
X = news_dataset.drop(columns = 'label', axis = 1)
Y = news_dataset['label']

## Stemming

Stemming is the process of reducing a word to its root word.

In [118]:
port_stem = PorterStemmer()

In [119]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # remove all the values in the content other than alphabets
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split() # converting to a list
    
    # stemming of all the words not in stopwords                                       
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    
    # join all the words
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [120]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [121]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [122]:
news_dataset['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

## Seperating the data

In [123]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [124]:
# converting the textual data to numeric form
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [125]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2, stratify = Y)

In [126]:
model = LogisticRegression()

In [128]:
model.fit(X_train, Y_train)

LogisticRegression()

In [130]:
X_test_prediction = model.predict(X_test)
accuracy = accuracy_score(X_test_prediction, Y_test)

In [131]:
print(accuracy)

0.9790865384615385
